In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.8MB/s 


In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/starter-dw"

/content/drive/My Drive/Colab Notebooks/starter-dw


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [5]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [7]:
df['brand_cat'] = df['brand'].factorize()[0]
run_model(['brand_cat'])

(-58.38655694633361, 4.223555478221712)

In [8]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.47223572384038, 4.328288468270897)

In [9]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [10]:
# Dodanie zmiany nazw Bradów na małe litery - dała lekką poprawę
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [11]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [12]:
# ciekawy parametr warty do sprawdzenia
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [13]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [0]:
def parse_features(x):
  if str(x) == 'nan' : return []
  return literal_eval(x.replace('\\"', '"'))

df['features_parsed'] = df['features'].map(parse_features)

In [15]:
df['features_parsed'].head().values

array([list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Shoe Size', 'value': ['M']}, {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, {'key': 'Color', 'value': ['Multicolor']}, {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, {'key': 'Brand', 'value': ['Josmo']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Shoe Size', 'value': ['M']}, {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, {'key': 'Color', 'value': ['Multicolor']}, {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, {'key': 'Brand', 'value': ['Josmo']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Color', 'value': ['Black']}, {'key': 'Shipping Weight (in pounds)', 'value': ['0.45']}, {'key': 'Condition', 'value': ['New']}, {'key': 'Brand', 'value': ['SERVUS BY HONEYWELL']}, {'key': 'manufacturer_part_number', 'value': ['ZSR101BLMLG']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Color', 'value': ['Black']}, {'key': 'Shipping Weight (in

In [0]:
# przygotowanie prostszej listy, wraz z małymi literami i usuwaniem spacji - do manewrowania danymi
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)


In [18]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [19]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()) )

len(keys)


476

In [20]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [21]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] =  df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)
  # sprawdzmy czy key istnieje w feats, jeżeli tak to zwracamy wartość feats, inaczej wstawiamy nan i w ten sposób mamy nowe kolumny

In [22]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_labor', 'feat_material composition',
       'feat_country of manufacturer', 'feat_color family', 'feat_dimensions',
       'feat_case material', 'feat_number of compartments',
       'feat_primary color', 'feat_feature', 'feat_fishing'],
      dtype='object', length=526)

In [23]:
df[ df['feat_athlete'].isnull() ].shape

(18272, 526)

In [24]:
df.shape[0]

18280

In [25]:
# wcześniej pokazywaliśmy ile razy nie występuje, a teraz ile razy jest
df[ False == df['feat_athlete'].isnull() ].shape


(8, 526)

In [26]:
# w ilu procent wypadkach ta cecha występuje
df[ False == df['feat_athlete'].isnull() ].shape[0] / df.shape[0] * 100

0.0437636761487965

In [0]:
keys_stat = {}
for key in keys:
 keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [28]:
keys_stat

{'100% adidas authentic': 0.005470459518599562,
 'accessory type': 0.1422319474835886,
 'adidas': 0.005470459518599562,
 'adjustable': 0.34463894967177244,
 'age': 0.6400437636761488,
 'age end': 0.4431072210065646,
 'age gender group': 0.12582056892778995,
 'age group': 27.64770240700219,
 'age range': 0.5470459518599562,
 'age segment': 0.1422319474835886,
 'age start': 0.4431072210065646,
 'airport friendly': 0.005470459518599562,
 'alarm': 0.23522975929978115,
 'amazonbestsellersrank': 0.06017505470459519,
 'animal type': 0.005470459518599562,
 'antiscratch lens coating': 0.005470459518599562,
 'applicable': 0.02188183807439825,
 'arm': 0.6236323851203501,
 'article': 0.005470459518599562,
 'assembled in country of origin': 2.199124726477024,
 'assembled product dimensions (l x w x h)': 10.300875273522976,
 'assembled product weight': 1.5536105032822756,
 'athlete': 0.0437636761487965,
 'atpv arc rating': 0.005470459518599562,
 'attachment': 0.03282275711159737,
 'audience': 0.2352

In [29]:
# 30 minuta
{k:v for k,v in keys_stat.items() if v > 0.5}

{'age': 0.6400437636761488,
 'age group': 27.64770240700219,
 'age range': 0.5470459518599562,
 'arm': 0.6236323851203501,
 'assembled in country of origin': 2.199124726477024,
 'assembled product dimensions (l x w x h)': 10.300875273522976,
 'assembled product weight': 1.5536105032822756,
 'battery type': 1.6083150984682715,
 'brand': 48.62691466083151,
 'bridge': 0.6236323851203501,
 'bridge size:': 0.5142231947483589,
 'bridge/temple size:': 0.5142231947483589,
 'case type:': 0.5142231947483589,
 'casual & dress shoe style': 12.910284463894966,
 'character': 0.5142231947483589,
 'clothing size': 1.7177242888402624,
 'color': 47.784463894967175,
 'condition': 18.12910284463895,
 'country of origin - assembly': 3.801969365426696,
 'country of origin - components': 4.091903719912473,
 'country/region of manufacture': 4.62253829321663,
 'designer': 1.1652078774617067,
 'ean': 1.2089715536105032,
 'eye size:': 0.5142231947483589,
 'fabric content': 6.083150984682714,
 'fabric material': 

In [32]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

In [35]:
df[ df.brand == df.feat_brand ].shape

(52, 531)

In [38]:

df[ df.brand != df.feat_brand ] [ ['brand', 'feat_brand'] ].head()

,brand,feat_brand
0,Josmo,josmo
1,Josmo,josmo
2,SERVUS BY HONEYWELL,servus by honeywell
3,SERVUS BY HONEYWELL,servus by honeywell
4,SERVUS BY HONEYWELL,servus by honeywell


In [39]:
df[ df.brand == df.feat_brand ] [ ['brand', 'feat_brand'] ].head()

,brand,feat_brand
2163,adidas,adidas
2417,adidas,adidas
3021,adidas,adidas
3054,adidas,adidas
3629,totes,totes


In [40]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[ df.brand != df.feat_brand ] [ ['brand', 'feat_brand'] ].head()

,brand,feat_brand
21,rubies,generic
22,rubies,generic
23,rubies,generic
24,unbranded,NaN
31,american fighter,NaN


In [41]:
df[ df.brand == df.feat_brand ].shape

(8846, 531)

In [43]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.338467995644166, 4.222571659281445)

In [44]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_manufacturer part number_cat', 'feat_material_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.17122927998662, 4.298079621665092)

In [46]:
X = df[ feats ].values
y = df[ 'prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2565 ± 0.0110,brand_cat
0.1057 ± 0.0095,feat_material_cat
0.0431 ± 0.0043,feat_gender_cat
0.0204 ± 0.0010,feat_brand_cat
0.0072 ± 0.0004,feat_color_cat
0.0008 ± 0.0001,feat_manufacturer part number_cat


In [47]:
#usuwamy najgorszą cechę
feats = ['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_material_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.113927578937385, 4.180794469672221)

In [48]:
X = df[ feats ].values
y = df[ 'prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2581 ± 0.0103,brand_cat
0.1015 ± 0.0115,feat_material_cat
0.0438 ± 0.0020,feat_gender_cat
0.0204 ± 0.0003,feat_brand_cat
0.0065 ± 0.0013,feat_color_cat


In [49]:
#usuwamy najgorszą cechę
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.10421508115066, 4.197320787726363)

In [50]:
df['brand'].value_counts()

nike            1777
puma             609
ralph lauren     526
vans             386
new balance      371
                ... 
hue                1
drmartens          1
hudson             1
mr.chaos           1
miller             1
Name: brand, Length: 1732, dtype: int64

In [51]:
df['brand'].value_counts(normalize=True)

nike            0.097210
puma            0.033315
ralph lauren    0.028775
vans            0.021116
new balance     0.020295
                  ...   
hue             0.000055
drmartens       0.000055
hudson          0.000055
mr.chaos        0.000055
miller          0.000055
Name: brand, Length: 1732, dtype: float64

In [53]:
df[ df['brand'] == 'nike' ].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

In [58]:
#dodajemy ciekawe cechy
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat', 'feat_style_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.09139951111494, 4.273503948690455)

In [59]:
df[ df['brand'] == 'nike' ].features_parsed.sample(5).values

array([{'style': 'athletic sneakers', 'pattern': 'usa', 'condition': 'new with box'},
       {'brand': 'nike', 'fabric content': '100% synthetic', 'color': 'white/bronze-red'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'color': 'dark pewter/black/grey/volt', 'casual & dress shoe style': 'athletic sneakers', 'manufacturer part number': '631754007', 'brand': 'nike', 'shoe width': 'medium (d, m)'},
       {'sport': 'soccer', 'condition': 'new with box', 'type': 'cleats'},
       {'manufacturer part number': 'nk 683895-071 g/w/g 13', 'gender': 'men', 'brand': 'nike', 'age group': 'adult', 'color': 'stlth/gld/wh/gra'}],
      dtype=object)

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [61]:
# automat robiąc wszystkie feat_..._cat
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_catalog',
 'feat_multi pack indicator',
 'feat_location - country',
 'feat_shoe category',
 'feat_clothing category',
 'feat_fabrication',
 'feat_recommended location',
 'feat_certifications and listings',
 'feat_location - city/state',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_uv protection_cat',
 'feat_removable hood_cat',
 'feat_motion control_cat',
 'feat_item weight_cat',
 'feat_interior pockets_cat',
 'feat_overall material_cat',
 'feat_vendor description_cat',
 'feat_clasp type_cat',
 'feat_number of heat settings_cat',
 'feat_wind resistant_cat',
 'feat_material type_cat',
 'feat_crown_cat',
 'feat_sku number_cat',
 'feat_catalog_cat',
 'feat_shade_cat',
 'feat_material detail-1_cat',
 'feat_cleaning, care & maintenance_cat',
 'feat_band material_cat',
 'feat_designed for_cat',
 'feat_type of sole_cat',
 'feat_vehicle type_

In [0]:
#dodajemy ciekawe cechy
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat', 'feat_style_cat']
feats += [ 'feat_polarised_cat',
 'feat_batteries included?_cat',
 'feat_dial material_cat',
 'feat_insulated_cat',
 'feat_is wheeled_cat',
 'feat_necklace type_cat',
 'feat_age group_cat',
 'feat_jacket length_cat',
 'feat_genre_cat',
 'feat_case tone_cat',
 'feat_features_cat',
 'feat_bracelet style_cat',
 'feat_frame shape_cat',
 'feat_age segment_cat',
 'feat_sleeve style_cat',
 'feat_diamond clarity_cat',
 'feat_age range_cat',
 'feat_материал_cat',
 'feat_sub type_cat',
 'feat_is lined_cat',
 'feat_navy blue_cat',
 'feat_protects against_cat',
 'feat_isbn_cat',
 'feat_video game platform_cat',
 'feat_band manufacturer_cat',
 'feat_cm_cat',
 'feat_finish_cat',
 'feat_manufacturer number_cat',
 'feat_closure style_cat',
 'feat_military style_cat',
 'feat_age_cat',
 'feat_looks best on these face shapes:_cat',
 'feat_temple size:_cat',
 'feat_antiscratch lens coating_cat',
 'feat_front style_cat',
 'feat_is orthopedic_cat',
 'feat_seller inventory number_cat',
 'feat_style code_cat',
 'feat_toe type_cat',
 'feat_style:_cat',
 'feat_reinforced pockets_cat',
 'feat_contained battery type_cat',
 'feat_assembled product weight_cat',
 'feat_hunting_cat',
 'feat_sole_cat',
 'feat_is dark sky-compliant_cat',
 'feat_lining material_cat',
 'feat_item width_cat',
 'feat_fits over eyeglasses_cat',
 'feat_fabric care_cat',
 'feat_domestic shipping_cat',
 'feat_box_cat',
 'feat_front_cat',
 'feat_item type_cat',
 'feat_sports team_cat',
 'feat_primary shelf id_cat',
 'feat_chest_cat',
 'feat_sku#_cat',
 'feat_best sellers rank_cat',
 'feat_lining_cat',
 'feat_count_cat',
 'feat_light weight_cat',
 'feat_eye size:_cat',
 'feat_parts_cat',
 'feat_removable liner_cat',
 'feat_has mercury_cat',
 'feat_chronograph_cat',
 'feat_is energy star-certified_cat',
 'feat_eyewear frame style_cat',
 'feat_authenticity_cat',
 'feat_lens technology_cat',
 'feat_movement_cat',
 'feat_neckline_cat',
 'feat_garment care_cat',
 'feat_applicable_cat',
 'feat_occasion_cat',
 'feat_net weight_cat',
 'feat_metal stamp_cat',
 'feat_manufacturer_part_number_cat',
 'feat_fit_cat',
 'feat_adjustable_cat',
 'feat_leather grade_cat',
 'feat_picture_cat',
 'feat_is waterproof_cat',
 'feat_polarized?_cat',
 'feat_lens tint_cat',
 'feat_lens type_cat',
 'feat_closure_cat',
 'feat_metal type_cat',
 'feat_black_cat',
 'feat_number of items_cat',
 'feat_general warranty_cat',
 'feat_package_cat',
 'feat_power reserve_cat',
 'feat_use_cat',
 'feat_polarized lenses_cat',
 'feat_sizearm_cat',
 'feat_resizable_cat',
 'feat_fastener type_cat',
 'feat_fabrication_cat',
 'feat_capacity_cat',
 'feat_airport friendly_cat',
 'feat_face shape_cat',
 'feat_clothing type_cat',
 'feat_lens material_cat',
 'feat_country////region of manufacture_cat',
 'feat_frame style_cat',
 'feat_bed size_cat',
 'feat_height_cat',
 'feat_has adaptive lenses_cat',
 'feat_compass_cat',
 'feat_number of wheels_cat',
 'feat_character_cat',
 'feat_bridge width_cat',
 'feat_licensed_cat',
 'feat_frame color_cat',
 'feat_inseam_cat',
 'feat_safety feature_cat',
 'feat_uv rating_cat',
 'feat_main color_cat',
 'feat_mirrored_cat',
 'feat_chest size_cat',
 'feat_dial markers_cat',
 'feat_amazonbestsellersrank_cat',
 'feat_lens socket width_cat',
 'feat_fits styles_cat',
 'feat_recommended location_cat',
 'feat_sports league_cat',
 'feat_nike id_cat',
 'feat_design_cat',
 'feat_shipping to usa_cat',
 'feat_arm_cat',
 'feat_resolution megapixels_cat',
 'feat_weight_cat',
 'feat_lined_cat',
 'feat_shirt size_cat',
 'feat_clothing size type_cat',
 'feat_safety apparel product type_cat',
 'feat_work shoes_cat',
 'feat_walmart no._cat',
 'feat_includes_cat',
 'feat_chain length_cat',
 'feat_manufacturer_cat',
 'feat_leg_cat',
 'feat_country of origin_cat',
 'feat_crystal_cat',
 'feat_heat zones_cat',
 'feat_series name_cat',
 'feat_charge time (minutes)_cat',
 'feat_sku_cat',
 'feat_product in inches (l x w x h)_cat',
 'feat_sizebridge_cat',
 'feat_euro size_cat',
 'feat_chain included_cat',
 'feat_tactical_cat',
 'feat_water resistant_cat',
 'feat_audience_cat',
 'feat_digital camera_cat',
 'feat_heart rate monitor_cat',
 'feat_issued/ not-issued_cat',
 'feat_pattern or design_cat',
 'feat_pant style_cat',
 'feat_lens_cat',
 'feat_flame resistant_cat',
 'feat_fabric care instructions_cat',
 'feat_alarm_cat',
 'feat_item package quantity_cat',
 'feat_batteries required_cat',
 'feat_length_cat',
 'feat_shoe closure_cat',
 'feat_colour code_cat',
 'feat_stock no._cat',
 'feat_model no._cat',
 'feat_global composite sports type_cat',
 'feat_autographed_cat',
 'feat_shipping weight_cat',
 'feat_assembled in country of origin_cat',
 'feat_pronation_cat',
 'feat_size_cat',
 'feat_fabric_cat',
 'feat_country of origin components:_cat',
 'feat_certifications and listings_cat',
 'feat_hood_cat',
 'feat_characteristics_cat',
 'feat_origin of components_cat',
 'feat_hardsided or softsided_cat',
 'feat_disclosures_cat',
 'feat_article_cat',
 'feat_pocket_cat',
 'feat_100% adidas authentic_cat',
 'feat_target audience_cat',
 'feat_weather-resistant_cat',
 'feat_thick, warm, cozy_cat',
 'feat_mpn_cat',
 'feat_spikes type_cat',
 'feat_date first available at amazon.co.uk_cat',
 'feat_shipping weight (in pounds)_cat',
 'feat_item color_cat',
 'feat_is recyclable_cat',
 'feat_leather :_cat',
 'feat_impact resistant_cat',
 'feat_frame material:_cat',
 'feat_insulation_cat',
 'feat_measurements:_cat',
 'feat_date first available at amazon.ca_cat',
 'feat_wheeled_cat',
 'feat_hairstyle_cat',
 'feat_item style_cat',
 'feat_frame depth_cat',
 'feat_comfort technology_cat',
 'feat_frame type_cat',
 'feat_case diameter_cat',
 'feat_ground_cat',
 'feat_metal color_cat',
 'feat_rise_cat',
 'feat_year_cat',
 'feat_made in_cat',
 'feat_attachment_cat',
 'feat_part number_cat',
 'feat_mpn#_cat',
 'feat_assembled product dimensions (l x w x h)_cat',
 'feat_item weight unit of measure_cat',
 'feat_materials_cat',
 'feat_casual & dress shoe style_cat',
 'feat_construction_cat',
 'feat_color/finish family_cat',
 'feat_fit:_cat',
 'feat_country of origin - assembly_cat',
 'feat_kids backpacks_cat',
 'feat_golf shoe type_cat',
 'feat_outer material_cat',
 'feat_power type_cat',
 'feat_fabric content_cat',
 'feat_sub style_cat',
 'feat_bridge_cat',
 'feat_szie_cat',
 'feat_age gender group_cat',
 'feat_adidas_cat',
 'feat_waterproof_cat',
 'feat_boxed-product dimensions_cat',
 'feat_number of pockets_cat',
 'feat_protection_cat',
 'feat_guaranteed authentic_cat',
 'feat_sock size_cat',
 'feat_msrp_cat',
 'feat_fits most screen size_cat',
 'feat_is weather-resistant_cat',
 'feat_purpose_cat',
 'feat_mechanic_cat',
 'feat_athlete_cat',
 'feat_label_cat',
 'feat_case type_cat',
 'feat_temple length_cat',
 'feat_handbag style_cat',
 'feat_theme_cat',
 'feat_transactionid_cat',
 'feat_safety features_cat',
 'feat_style #_cat',
 'feat_place of origin_cat',
 'feat_country/region of manufacture_cat',
 'feat_auto_cat',
 'feat_recommended use_cat',
 'feat_is portable_cat',
 'feat_battery type_cat',
 'feat_case type:_cat',
 'feat_watch shape_cat',
 'feat_resolution_cat',
 'feat_carats_cat',
 'feat_nike product #_cat',
 'feat_ean_cat',
 'feat_bridge/temple size:_cat',
 'feat_water-resistant_cat',
 'feat_to fit_cat',
 'feat_item spec attr - style (ebay)_cat',
 'feat_date first available_cat',
 'feat_product dimensions_cat',
 'feat_diamond color_cat',
 'feat_clothing product type_cat',
 'feat_leg opening:_cat',
 'feat_machine washable_cat',
 'feat_nickel free_cat',
 'feat_quantity in set_cat',
 'feat_technician_cat',
 'feat_manufacturer sku_cat',
 'feat_shape_cat',
 'feat_age end_cat',
 'feat_gold karat_cat',
 'feat_upper material_cat',
 'feat_gps_cat',
 'feat_sock style_cat',
 'feat_country of origin assembly:_cat',
 'feat_chain/necklace length (in.)_cat',
 'feat_model number_cat',
 'feat_fits model_cat',
 'feat_lens material:_cat',
 'feat_name_cat',
 'feat_cushioning_cat',
 'feat_year made_cat',
 'feat_case thickness_cat',
 'feat_stylenumber_cat',
 'feat_location - city/state_cat',
 'feat_case back_cat',
 'feat_country/ organization_cat',
 'feat_hat style_cat',
 'feat_style number_cat',
 'feat_productgroup_cat',
 'feat_country//region of manufacture_cat',
 'feat_jewelry setting_cat',
 'feat_atpv arc rating_cat',
 'feat_sleeve length_cat',
 'feat_ul safety listing_cat',
 'feat_watch band material_cat',
 'feat_product name_cat',
 'feat_style name_cat',
 'feat_shoe size_cat',
 'feat_special features_cat',
 'feat_inv_cat',
 'feat_fine or fashion_cat',
 'feat_case finish_cat',
 'feat_bag size range_cat',
 'feat_frame material_cat',
 'feat_fastener_cat',
 'feat_accessory type_cat',
 'feat_polarized_cat',
 'feat_bridge size:_cat',
 'feat_product id_cat',
 'feat_international shipping_cat',
 'feat_date_cat',
 'feat_season_cat',
 'feat_fits brands_cat',
 'feat_type 2_cat',
 'feat_very popular bag now a days_cat',
 'feat_weather resistant_cat',
 'feat_is polarized_cat',
 'feat_animal type_cat',
 'feat_bezel_cat',
 'feat_auto shut-off_cat',
 'feat_high visibility (ansi compliant)_cat',
 'feat_����������������_cat',
 'feat_wheel type_cat',
 'feat_charger included_cat',
 'feat_footwear type_cat',
 'feat_used with equipment type_cat',
 'feat_suitable for_cat',
 'feat_has paper wood_cat',
 'feat_band length_cat',
 'feat_labor_cat',
 'feat_material composition_cat',
 'feat_country of manufacturer_cat',
 'feat_color family_cat',
 'feat_dimensions_cat',
 'feat_case material_cat',
 'feat_number of compartments_cat',
 'feat_primary color_cat',
 'feat_feature_cat',
 'feat_fishing_cat']


model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [66]:
X = df[ feats ].values
y = df[ 'prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

(-57.44274993771198, 4.311353667525453)


Weight,Feature
0.2489 ± 0.0094,brand_cat
0.1100 ± 0.0052,feat_material_cat
0.0194 ± 0.0017,feat_brand_cat
0.0154 ± 0.0019,feat_fabric content_cat
0.0124 ± 0.0010,feat_weight_cat
0.0089 ± 0.0028,feat_adjustable_cat
0.0067 ± 0.0016,feat_resizable_cat
0.0063 ± 0.0007,feat_case thickness_cat
0.0040 ± 0.0006,feat_gender_cat
0.0034 ± 0.0003,feat_jacket length_cat


In [0]:
!git add starterone/StarterDW_day5.ipynb

In [0]:
!git commit -m "Model - Men's Shoe Prices dataset from data.world"